In [1]:
import pandas as pd 

In [2]:
year = '2016'

In [3]:
df = pd.read_csv('points_against_'+year+'.csv')

In [4]:
# Calculate Team Against Ave/Std
for pos in df.Position.unique():
    for team in df.Team_Against.unique():
        num_weeks = df[(df.Team_Against == team) & (df.Position == pos)].Game_Count.sum()
        if num_weeks != 0:
            ave = df[(df.Team_Against == team) & (df.Position == pos)].Points_Against.astype(float).sum() / num_weeks
        else:
            print(df[(df.Team_Against == team) & (df.Position == pos)])
        std = df[(df.Team_Against == team) & (df.Position == pos)].Points_Against.astype(float).std()
        df.loc[(df.Team_Against == team) & (df.Position == pos), 'Against_Ave'] = ave
        df.loc[(df.Team_Against == team) & (df.Position == pos), 'Against_Std'] = std
        df['Against_16th'] = df['Against_Ave'] - df['Against_Std']
        df['Against_84th'] = df['Against_Ave'] + df['Against_Std']

In [5]:
# Calculate Deviation from average
df['Game_Dev'] = (df.Points_Against - df.Against_Ave)/(df.Against_Std)

In [6]:
# Construct Summary DF from the 'VS' team perspective
df_vs = df[['VS','Position','Year']]
df_vs = df_vs.drop_duplicates()
df_vs = df_vs[df_vs.VS != 'Bye']

In [7]:
# Calculate Ave/Std (of deviations) for the VS teams
df_vs['VS_Ave'] = 0
df_vs['VS_Std'] = 0
for index, row in df_vs.iterrows():
    df_vs.loc[(df_vs['VS'] == row.VS) & (df_vs.Position == row.Position) & (df_vs.Year == row.Year),'VS_Ave'] = df[(df.VS == row.VS) & (df.Position == row.Position) & (df.Year == row.Year)].Game_Dev.mean()
    df_vs.loc[(df_vs['VS'] == row.VS) & (df_vs.Position == row.Position) & (df_vs.Year == row.Year),'VS_Std'] = df[(df.VS == row.VS) & (df.Position == row.Position) & (df.Year == row.Year)].Game_Dev.std()

In [8]:
# Predict scores in a matchup
def matchup(team_against, vs):
    df_against = df[(df['Team_Against'] == team_against)][['Team_Against','Position','Against_Ave','Against_Std','Against_16th','Against_84th']]
    df_against = df_against.drop_duplicates()
    
    df_matchup = pd.merge(left=df_against, right=df_vs[df_vs['VS']==vs], left_on='Position', right_on='Position')
    
    df_matchup['Weighted_16th'] = (df_matchup['VS_Ave']+1-df_matchup['VS_Std'])*df_matchup['Against_Std']
    df_matchup['Weighted_Mean'] = (df_matchup['VS_Ave']+1)*df_matchup['Against_Std']
    df_matchup['Weighted_84th'] = (df_matchup['VS_Ave']+1+df_matchup['VS_Std'])*df_matchup['Against_Std']
    
    return df_matchup
    

In [9]:
matchup('HOU', 'NE')

,Team_Against,Position,Against_Ave,Against_Std,Against_16th,Against_84th,VS,Year,VS_Ave,VS_Std,Weighted_16th,Weighted_Mean,Weighted_84th
0,HOU,QB,13.97250,6.375006,7.597494,20.347506,NE,2016,0.467507,0.840003,4.000338,9.355365,14.710393
1,HOU,RB,17.65000,8.974121,8.675879,26.624121,NE,2016,0.455752,0.720619,6.597169,13.064094,19.531019
2,HOU,WR,19.05625,9.893555,9.162695,28.949805,NE,2016,-0.103274,1.041013,-1.427518,8.871806,19.171130
3,HOU,TE,4.74375,4.357026,0.386724,9.100776,NE,2016,0.644127,1.290931,1.538885,7.163503,12.788122
4,HOU,K,7.25000,3.626901,3.623099,10.876901,NE,2016,0.256299,0.889836,1.329122,4.556471,7.783820
5,HOU,DEF,8.37500,4.973459,3.401541,13.348459,NE,2016,0.034628,0.796084,1.186390,5.145680,9.104970


In [10]:
df[(df['VS'] == 'HOU')&(df['Team_Against']=='NE')]

,Unnamed: 0,Game_Count,Points_Against,Position,Team_Against,VS,Week,Year,Home_Away,Against_Ave,Against_Std,Against_16th,Against_84th,Game_Dev
90,0,1,7.14,QB,NE,HOU,3,2016,Home,15.62625,7.785380,7.840870,23.411630,-1.090024
629,0,1,11.80,RB,NE,HOU,3,2016,Home,14.45625,7.378641,7.077609,21.834891,-0.359992
1179,0,1,10.20,WR,NE,HOU,3,2016,Home,20.68125,10.519680,10.161570,31.200930,-0.996347
1712,0,1,5.20,TE,NE,HOU,3,2016,Home,5.67500,4.197180,1.477820,9.872180,-0.113171
2271,0,1,0.00,K,NE,HOU,3,2016,Home,5.62500,4.104338,1.520662,9.729338,-1.370501
2811,0,1,1.00,DEF,NE,HOU,3,2016,Home,2.50000,4.358055,-1.858055,6.858055,-0.344190
